In [1]:
import pandas as pd
pd.set_option('display.max_columns',500)
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import lxml

In [3]:
bs4.builder.builder_registry.builders

[bs4.builder._lxml.LXMLTreeBuilder,
 bs4.builder._lxml.LXMLTreeBuilderForXML,
 bs4.builder._html5lib.HTML5TreeBuilder,
 bs4.builder._htmlparser.HTMLParserTreeBuilder]

In [4]:
import re

In [4]:
import sqlite3
conn = sqlite3.connect("/home/ubuntu/FPA_FOD_20170508.sqlite")

In [9]:
fires = pd.read_sql_query('SELECT * from fires', conn)

In [15]:
fires.shape

(1880465, 39)

In [17]:
fires.sample(100)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
279076,279077,284458,W-586953,FED,DOI-WFMI,BIA,USORWSA,Warm Springs Agency,ORWSA,Warm Springs Agency,7,None,EW8B,MILL,None,None,None,None,None,2009,2454974.5,143,2015,9.0,Miscellaneous,2454974.5,143.0,2130,0.10,A,44.768000,-121.240300,2.0,BIA,OR,None,None,None,b'\x00\x01\xad\x10\x00\x00\xa4N@\x13aO^\xc0\x0...
1127233,1127234,1373849,CDF_1993_55_2221_11685,NONFED,ST-CACDF,ST/C&L,USCABDU,San Bernardino Unit,CABDU,CDF - San Bernardino Unit,None,11685,None,ELMWOOD,None,None,None,None,None,1993,2449296.5,309,None,9.0,Miscellaneous,NaN,NaN,None,0.10,A,34.128889,-117.251944,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00`\x8d\x92\xdb\x1fP]\...
1146490,1146491,1397258,ODF-86801,NONFED,ST-ORORS,ST/C&L,USOR981S,Klamath Unit,ORODF-981,Klamath Unit,030,86801,None,JUNIPER,None,None,None,None,None,2010,2455403.5,207,None,1.0,Lightning,2455403.5,207.0,None,0.10,A,42.358330,-121.521940,8.0,PRIVATE,OR,18,035,Klamath,b'\x00\x01\xad\x10\x00\x00X\x9e\x07wga^\xc0\x8...
264973,264974,270022,W-522088,FED,DOI-WFMI,BIA,USSDPRA,Pine Ridge Agency,SDPRA,Pine Ridge Agency,255,None,CRV4,COHEN HOME,None,None,None,None,None,2006,2453909.5,174,1239,10.0,Fireworks,2453909.5,174.0,1255,0.50,B,43.032200,-102.557200,2.0,BIA,SD,None,None,None,b'\x00\x01\xad\x10\x00\x000U0*\xa9\xa3Y\xc08w-...
608760,608761,656641,SFO-TX01430696-10440094,NONFED,ST-NASF,ST/C&L,USTXHAS,Texas Forest Service - Henderson Area,TXHAS,Texas Forest Service - Henderson Area,None,None,None,None,None,None,None,None,None,1996,2450091.5,9,None,7.0,Arson,NaN,NaN,None,7.00,B,31.877080,-95.964580,14.0,MISSING/NOT SPECIFIED,TX,Anderson,001,Anderson,b'\x00\x01\xad\x10\x00\x00\x08\x98\xc0\xad\xbb...
604665,604666,652511,SFO-TX01430692-35731976,NONFED,ST-NASF,ST/C&L,USTXWZS,Texas Forest Service - West Zone,TXWZS,Texas Forest Service - West Zone,None,None,None,None,None,None,None,None,None,1992,2448871.5,250,None,5.0,Debris Burning,NaN,NaN,None,1.00,B,29.989580,-97.060420,14.0,MISSING/NOT SPECIFIED,TX,Fayette,149,Fayette,b'\x00\x01\xad\x10\x00\x00\x8c\x01\xd9\xeb\xdd...
161551,161552,163125,FS-391114,FED,FS-FIRESTAT,FS,USAZASF,Apache-Sitgreaves National Forest,0301,Apache-Sitgreaves National Forest,197,None,5555,KP,None,None,None,None,None,2003,2452841.5,202,1900,1.0,Lightning,2452872.5,233.0,1900,10.00,C,33.583611,-109.315000,5.0,USFS,AZ,11,011,Greenlee,b'\x00\x01\xad\x10\x00\x00X\x8f\xc2\xf5(T[\xc0...
658132,658133,714324,SFO-TX0484-186731,NONFED,ST-NASF,ST/C&L,USTXTXS,Texas A & M Forest Service,TXTXS,Texas Forest Service,None,None,None,None,None,None,None,None,None,2008,2454619.5,154,None,5.0,Debris Burning,NaN,NaN,None,10.00,C,31.355340,-98.087940,14.0,MISSING/NOT SPECIFIED,TX,Coryell,099,Coryell,b'\x00\x01\xad\x10\x00\x00\xa4\x00\x18\xcf\xa0...
194392,194393,197025,W-361976,FED,DOI-WFMI,BIA,USMTFBA,Fort Belknap Agency,MTFBA,Fort Belknap Agency,44,None,None,ANTLER,None,None,None,None,None,1997,2450644.5,196,2330,10.0,Fireworks,2450645.5,197.0,0030,0.10,A,47.983300,-108.700700,8.0,PRIVATE,MT,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xa8\x13\xd0D\xd8,[\..."
819050,819051,926212,STATE_MS_66242,NONFED,ST-MSMSS,ST/C&L,USMSMSS,Mississippi Forestry Commission,MS Southeast,MSS Southeast District,None,3-4373,None,None,None,None,None,None,None,2006,2453779.5,44,None,7.0,Arson,NaN,NaN,None,1.00,B,31.279700,-89.370800,14.0,MISSING/NOT SPECIFIED,MS,None,None,None,b'\x00\x01\xad\x10\x00\x00\xd4V\xec/\xbbWV\xc0...


## Apply cleaning

In [38]:
fires = fires[['FOD_ID', 'SOURCE_SYSTEM_TYPE', 'NWCG_REPORTING_AGENCY', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'STATE', 'COUNTY', 'FIPS_CODE']]
#drop probably useless/unknown columns

In [39]:
fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'], unit='D', origin='julian') #julian dates
fires['Month'] = fires['DISCOVERY_DATE'].dt.month
fires['DayofWeek'] = fires['DISCOVERY_DATE'].dt.weekday_name
fires.head()

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek
0,1,FED,FS,2005,2005-02-02,33,1300,9.0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,63,063,2,Wednesday
1,2,FED,FS,2004,2004-05-12,133,0845,1.0,Lightning,0.25,A,38.933056,-120.404444,CA,61,061,5,Wednesday
2,3,FED,FS,2004,2004-05-31,152,1921,5.0,Debris Burning,0.10,A,38.984167,-120.735556,CA,17,017,5,Monday
3,4,FED,FS,2004,2004-06-28,180,1600,1.0,Lightning,0.10,A,38.559167,-119.913333,CA,3,003,6,Monday
4,5,FED,FS,2004,2004-06-28,180,1600,1.0,Lightning,0.10,A,38.559167,-119.933056,CA,3,003,6,Monday


In [40]:
%time fires['DISCOVERY_TIME2'] = fires['DISCOVERY_TIME'].astype('float64')

CPU times: user 152 ms, sys: 4 ms, total: 156 ms
Wall time: 153 ms


In [45]:
fires.sort_values(by='DISCOVERY_DATE', inplace=True)

In [46]:
fires.head()

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2
774670,878199,NONFED,ST/C&L,1992,1992-01-01,1,None,13.0,Missing/Undefined,0.25,A,41.642490,-108.245102,WY,Sweetwater,037,1,Wednesday,NaN
961584,1089849,NONFED,ST/C&L,1992,1992-01-01,1,None,6.0,Railroad,8.00,B,28.300000,-80.730000,FL,Brevard,009,1,Wednesday,NaN
961583,1089848,NONFED,ST/C&L,1992,1992-01-01,1,None,6.0,Railroad,10.00,C,25.950000,-80.450000,FL,Miami Dade,086,1,Wednesday,NaN
603990,651826,NONFED,ST/C&L,1992,1992-01-01,1,None,5.0,Debris Burning,5.00,B,31.664580,-94.589580,TX,Nacogdoches,347,1,Wednesday,NaN
746120,845656,NONFED,ST/C&L,1992,1992-01-01,1,None,13.0,Missing/Undefined,4.00,B,33.379167,-80.620833,SC,38,075,1,Wednesday,NaN


In [5]:
with open('clean_fires.pkl', 'rb') as picklefile:
    fires = pickle.load(picklefile)

## Fill in county names

In [6]:
def fill_county(county, lat, lon):
    c=0
    t=0
#     print(county)
#     print(type(county))
#     print(lat,lon)
#     print(type(lat), type(lon))
    if county is None:
        county = '0'
#         print(county)
#         print(len(county))
#         print(county.isdigit())
    if ((len(county) < 3) | (county.isdigit())): #fill only rows that need them
        try:
            url = 'https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}&showall=true&format=xml'.format(str(lat),str(lon))
#             print(url)
            soup = BeautifulSoup(requests.get(url).text, features="xml")
            tag = soup.find('County')
            return pd.Series((tag['name'].lower(), tag['FIPS']))
        except ValueError:
            c += 1
            print('c_error%s' % str(c))
            return pd.Series((county, county))
        except TypeError:
            t +=1
            print('t_error%s' % str(c))
            return pd.Series((county, county))
    else:
        return pd.Series((county.lower(), county)) 


In [7]:
def test_fill_county(county, lat, lon):
    c=0
    t=0
#     print(county)
#     print(type(county))
#     print(lat,lon)
#     print(type(lat), type(lon))
    if county is None:
        county = '0'
#         print(county)
#         print(len(county))
#         print(county.isdigit())
    if ((len(county) < 3) | (county.isdigit())): #fill only rows that need them
        #try:
            url = 'https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}&showall=true&format=xml'.format(str(lat),str(lon))
#             print(url)
            soup = BeautifulSoup(requests.get(url).text, features="xml")
            tag = soup.find('County')
            return pd.Series((tag['name'].lower(), tag['FIPS']))
            #or just #row['full_FIPS_CODE'] = tag['FIPS']
        #except ValueError:
#             c += 1
#             print('c_error%s' % str(c))
#             return pd.Series((county, county))
#         #except TypeError:
#             t +=1
#             print('t_error%s' % str(c))
#             return pd.Series((county, county)
    else:
        return pd.Series((county.lower(), county)) 


In [14]:
test_fill_county(fires.iloc[75448,-5], fires.iloc[75448, -8], fires.iloc[75448, -7])

0    covington
1    Covington
dtype: object

In [ ]:
https://geo.fcc.gov/api/census/block/find?latitude=31.6079&longitude=-89.6723&showall=true&format=xml

In [11]:
fires.iloc[75448,:]

FOD_ID                                769369
SOURCE_SYSTEM_TYPE                    NONFED
NWCG_REPORTING_AGENCY                 ST/C&L
FIRE_YEAR                               1993
DISCOVERY_DATE           1993-03-09 00:00:00
DISCOVERY_DOY                             68
DISCOVERY_TIME                          1430
STAT_CAUSE_CODE                            7
STAT_CAUSE_DESCR                       Arson
FIRE_SIZE                                  1
FIRE_SIZE_CLASS                            B
LATITUDE                             31.6079
LONGITUDE                           -89.6723
STATE                                     MS
COUNTY                             Covington
FIPS_CODE                                031
Month                                      3
DayofWeek                            Tuesday
DISCOVERY_TIME2                         1430
Name: 691664, dtype: object

In [8]:
fires[['COUNTY2', 'COUNTY_ID']] = fires.apply(lambda row: fill_county(row['COUNTY'], row['LATITUDE'], row['LONGITUDE']), axis=1)

In [10]:
with open('fires_w_county.pkl', 'wb') as picklefile:
    pickle.dump(fires, picklefile)

In [11]:
fires.sample(30)

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID
1703724,201921731,INTERAGCY,IA,2004,2004-02-03,34,None,13.0,Missing/Undefined,0.50,B,18.322454,-66.392960,PR,None,None,2,Tuesday,NaN,morovis,72101
1798146,300210337,FED,NPS,2015,2015-07-06,187,1608,1.0,Lightning,0.10,A,37.154900,-108.433300,CO,None,None,7,Monday,1608.0,montezuma,08083
818867,926025,NONFED,ST/C&L,2006,2006-04-12,102,None,9.0,Miscellaneous,3.00,B,30.821700,-89.508100,MS,None,None,4,Wednesday,NaN,pearl river,28109
1818427,300236561,NONFED,ST/C&L,2015,2015-01-20,20,1420,5.0,Debris Burning,5.00,B,35.142230,-90.773530,AR,None,None,1,Tuesday,1420.0,st. francis,05123
1781778,300137306,NONFED,ST/C&L,2014,2014-03-23,82,1301,9.0,Miscellaneous,0.10,A,40.737900,-73.130100,NY,SUFFOLK,103,3,Sunday,1301.0,suffolk,SUFFOLK
1810537,300225981,NONFED,ST/C&L,2015,2015-03-23,82,1510,2.0,Equipment Use,3.60,B,38.113424,-82.552806,WV,Wayne,099,3,Monday,1510.0,wayne,Wayne
899452,1025801,NONFED,ST/C&L,2002,2002-02-18,49,None,1.0,Lightning,5.00,B,35.763300,-94.876700,OK,None,None,2,Monday,NaN,cherokee,40021
1180618,1431814,FED,FS,2010,2010-09-20,263,1129,1.0,Lightning,0.10,A,44.538611,-121.605833,OR,031,031,9,Monday,1129.0,jefferson,41031
806699,913508,NONFED,ST/C&L,2007,2007-09-16,259,None,2.0,Equipment Use,0.60,B,44.632780,-117.778610,OR,1,001,9,Sunday,NaN,baker,41001
1600920,201620017,NONFED,ST/C&L,2012,2012-06-27,179,None,9.0,Miscellaneous,5.00,B,39.041978,-82.693740,OH,JACKSON,079,6,Wednesday,NaN,jackson,JACKSON


In [100]:
work_samp = fires.sample(10000)

In [13]:
with open('fire_10000.pkl', 'wb') as picklefile:
    pickle.dump(work_samp, picklefile)

## Try to create county ID instead of groupby State and County (come back to later if needed)

In [65]:
from urllib.request import urlopen
cols = ['STATE', 'StateID', 'FIPS_CODE', 'COUNTY2', 'FPClass']
url ='https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt'
county_list = pd.read_csv(url, header=None, sep=',', names=cols, dtype='str')

In [66]:
county_list['COUNTY2'] = county_list['COUNTY2'].str.replace(' County', '', regex=True) #safer

In [57]:
# county_list['COUNTY2'] = county_list['COUNTY2'].str[:-7] #remove County from the end 

In [67]:
county_list['COUNTY2'] = county_list['COUNTY2'].str.lower()

In [68]:
county_list[county_list['COUNTY2'] == 'powder river']

,STATE,StateID,FIPS_CODE,COUNTY2,FPClass
1635,MT,30,075,powder river,H1


In [69]:
add_county = work_samp.merge(county_list, how='left', on=['STATE','COUNTY2'])

In [70]:
add_county.sample(20)

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE_x,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,StateID,FIPS_CODE_y,FPClass
2885,1802575,NONFED,ST/C&L,1994,1994-04-21,111,1030,7.0,Arson,2.00,B,38.032145,-82.115595,WV,Lincoln,043,4,Thursday,1030.0,lincoln,Lincoln,54,043,H1
6099,1029299,NONFED,ST/C&L,1999,1999-11-13,317,None,8.0,Children,5.00,B,36.169200,-94.574400,OK,None,None,11,Saturday,NaN,delaware,40041,40,041,H1
2978,219420,FED,BLM,1995,1995-08-06,218,1042,5.0,Debris Burning,3.00,B,34.743600,-117.200900,CA,None,None,8,Sunday,1042.0,san bernardino,06071,06,071,H1
7687,1369619,NONFED,ST/C&L,1993,1993-11-27,331,None,5.0,Debris Burning,5.00,B,40.165000,-122.303056,CA,None,None,11,Saturday,NaN,tehama,06103,06,103,H1
8387,1134438,NONFED,ST/C&L,1993,1993-09-15,258,None,5.0,Debris Burning,3.00,B,35.545000,-78.125000,NC,None,None,9,Wednesday,NaN,johnston,37101,37,101,H1
8774,19093378,NONFED,ST/C&L,1992,1992-02-06,37,1245,5.0,Debris Burning,0.83,B,34.772600,-83.865900,GA,Union,291,2,Thursday,1245.0,union,Union,NaN,NaN,NaN
7492,300129696,NONFED,ST/C&L,2014,2014-03-17,76,1159,9.0,Miscellaneous,0.10,A,40.670800,-73.893800,NY,KINGS,047,3,Monday,1159.0,kings,KINGS,36,047,H6
9109,201389506,NONFED,ST/C&L,2009,2009-04-07,97,None,5.0,Debris Burning,1.00,B,33.531085,-94.637097,TX,Bowie,037,4,Tuesday,NaN,bowie,Bowie,48,037,H1
2817,361523,FED,NPS,2006,2006-09-02,245,1500,1.0,Lightning,0.10,A,37.151700,-108.488100,CO,None,None,9,Saturday,1500.0,montezuma,08083,08,083,H1
8447,412024,NONFED,ST/C&L,2005,2005-04-27,117,None,9.0,Miscellaneous,2.00,B,33.309370,-92.706450,AR,Union,139,4,Wednesday,NaN,union,Union,05,139,H1


## Consolidate causes

In [101]:
def set_label(cat):
    cause = 0
    Lightning = ['Lightning']
    accidental = ['Structure','Fireworks','Powerline','Railroad','Smoking','Children','Campfire','Equipment Use','Debris Burning']
    malicious = ['Arson']
    other = ['Missing/Undefined','Miscellaneous']
    if cat in Lightning:
        cause = 'Lightning'
    elif cat in accidental:
        cause = 'Accident'
    elif cat in malicious:
        cause = 'Arson'
    else:
        cause = 'Other'
    return cause
     


In [102]:
%time work_samp['Cause'] = work_samp['STAT_CAUSE_DESCR'].apply(lambda x:set_label(x))
work_samp.head(10)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 9.98 ms


,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause
767380,869320,NONFED,ST/C&L,1994,1994-09-17,260,None,5.0,Debris Burning,0.30,B,47.847840,-121.667290,WA,Snohomish County,061,9,Saturday,NaN,snohomish county,Snohomish County,Accident
332278,340552,FED,BLM,2000,2000-08-13,226,1138,2.0,Equipment Use,1.00,B,39.134900,-119.778500,NV,Washoe,031,8,Sunday,1138.0,washoe,Washoe,Accident
1316189,1794973,NONFED,ST/C&L,2003,2003-10-20,293,1622,5.0,Debris Burning,6.00,B,37.623844,-81.768897,WV,Wyoming,109,10,Monday,1622.0,wyoming,Wyoming,Accident
803434,909818,NONFED,ST/C&L,2000,2000-03-07,67,None,9.0,Miscellaneous,172.00,D,36.621100,-84.042800,KY,None,None,3,Tuesday,NaN,whitley,21235,Other
76012,76898,FED,FS,1995,1995-07-31,212,1400,1.0,Lightning,0.10,A,47.851667,-115.196667,MT,None,None,7,Monday,1400.0,sanders,30089,Lightning
1158462,1409449,NONFED,ST/C&L,2004,2004-04-07,98,1320,5.0,Debris Burning,1.50,B,33.116392,-86.252077,AL,Talladega,121,4,Wednesday,1320.0,talladega,Talladega,Accident
458474,492792,NONFED,ST/C&L,2009,2009-02-23,54,None,5.0,Debris Burning,5.92,B,30.869380,-84.546248,GA,Decatur,087,2,Monday,NaN,decatur,Decatur,Accident
217340,220550,FED,BLM,1998,1998-10-05,278,1833,9.0,Miscellaneous,0.10,A,33.881200,-116.501869,CA,Riverside,065,10,Monday,1833.0,riverside,Riverside,Other
73527,74359,FED,FS,1994,1994-07-23,204,1950,1.0,Lightning,0.20,A,48.877778,-117.247778,WA,None,None,7,Saturday,1950.0,pend oreille,53051,Lightning
1660869,201820773,NONFED,ST/C&L,2013,2013-03-31,90,0135,9.0,Miscellaneous,0.10,A,38.428200,-105.211800,CO,Fremont,043,3,Sunday,135.0,fremont,Fremont,Other


## Previous fires w/ condition: use for things that you don't know at time of discovery (cause, size)

In [103]:
#use
#generalize 
def prev_fires_filter(group, time_col, value_col, condition, new_col_name, year_interval=5):
    for idx, row in group.iterrows():
        past = (group[time_col] < row[time_col]) & (group[value_col] == condition)
        df = group[past] #make dataframe of past fires relative to current observation
        
        #df['yr_intervals'] = (row[time_col] - df[time_col]) // year_interval #if using 'FIRE_YEAR' col
        #print(df['yr_intervals'])
        df['yr_intervals'] = (row[time_col] - df[time_col]) // np.timedelta64(year_interval, 'Y') #how many 5-years have passed?
        #print(df['yr_intervals'])
        df['weight'] = 1 / 2**(df['yr_intervals'])
                #weight of 1 for within past 5 years, 0.5 for 5-10 years, 0.25 for 10-15 years, etc
        #df['prev_fires'] = df['weight'].sum()
        #print(df['weight'])
        count_fires = df['weight'].sum()
        #print(count_fires)
        #count_fires = df['prev_fires'].iloc[0]
        group.loc[idx,new_col_name] = count_fires
    return group
        

In [92]:
%time grouped_work_samp = work_samp.groupby(['STATE', 'COUNTY2'])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 248 µs


In [104]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Lightning', 'Prev_Lightning_Fires')

CPU times: user 54.1 s, sys: 0 ns, total: 54.1 s
Wall time: 54.1 s


In [105]:
work_samp.head()

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause,Prev_Lightning_Fires
767380,869320,NONFED,ST/C&L,1994,1994-09-17,260,None,5.0,Debris Burning,0.3,B,47.847840,-121.667290,WA,Snohomish County,061,9,Saturday,NaN,snohomish county,Snohomish County,Accident,0.0
332278,340552,FED,BLM,2000,2000-08-13,226,1138,2.0,Equipment Use,1.0,B,39.134900,-119.778500,NV,Washoe,031,8,Sunday,1138.0,washoe,Washoe,Accident,1.0
1316189,1794973,NONFED,ST/C&L,2003,2003-10-20,293,1622,5.0,Debris Burning,6.0,B,37.623844,-81.768897,WV,Wyoming,109,10,Monday,1622.0,wyoming,Wyoming,Accident,0.0
803434,909818,NONFED,ST/C&L,2000,2000-03-07,67,None,9.0,Miscellaneous,172.0,D,36.621100,-84.042800,KY,None,None,3,Tuesday,NaN,whitley,21235,Other,0.0
76012,76898,FED,FS,1995,1995-07-31,212,1400,1.0,Lightning,0.1,A,47.851667,-115.196667,MT,None,None,7,Monday,1400.0,sanders,30089,Lightning,0.0


In [ ]:
# df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)

In [97]:
# grouped_work_samp.apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Lightning', 'Prev_Lightning_Fires').values

array([array([0.]), array([0.]),
       array([1.  , 0.  , 0.25, 0.5 , 0.  , 0.25, 0.5 , 0.5 , 0.  ]), ...,
       array([0.5, 0. ]), array([0. , 1. , 0.5]), array([1., 0., 0.])],
      dtype=object)

In [ ]:
#grouped_work_samp.apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Lightning', 'Prev_Lightning_Fires'))
#....apply(y) work_samp['Prev_Lightning_Fires'] = 

In [83]:
# #Natural causes
# %time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Lightning', 'Prev_Natural_Fires')

CPU times: user 53.2 s, sys: 12 ms, total: 53.2 s
Wall time: 53.2 s


,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause,Prev_Natural_Fires
1290261,1761023,NONFED,ST/C&L,2009,2009-11-07,311,1237,9.0,Miscellaneous,2.50,B,41.196610,-73.970238,NY,ROCKLAND,087,11,Saturday,1237.0,rockland,ROCKLAND,Other,0.000
1173482,1424595,NONFED,ST/C&L,1999,1999-03-25,84,1330,5.0,Debris Burning,8.00,B,34.055426,-89.621093,MS,Yalobusha,161,3,Thursday,1330.0,yalobusha,Yalobusha,Accident,0.000
1880268,300347153,NONFED,ST/C&L,2012,2012-06-18,170,1107,3.0,Smoking,0.50,B,34.850000,-118.316666,CA,None,None,6,Monday,1107.0,kern,06029,Accident,1.500
566216,610570,NONFED,ST/C&L,2005,2005-10-30,303,None,13.0,Missing/Undefined,5.00,B,34.833700,-94.945700,OK,Le Flore,079,10,Sunday,NaN,le flore,Le Flore,Other,0.000
1706312,201924319,INTERAGCY,IA,2005,2005-02-15,46,None,13.0,Missing/Undefined,5.00,B,17.958364,-66.172852,PR,None,None,2,Tuesday,NaN,guayama,72057,Other,0.000
1222759,1602445,NONFED,ST/C&L,2010,2010-10-26,299,0044,13.0,Missing/Undefined,3.00,B,31.610896,-90.262729,MS,Lincoln,085,10,Tuesday,44.0,lincoln,Lincoln,Other,0.000
1028477,1158116,NONFED,ST/C&L,1998,1998-11-27,331,None,7.0,Arson,2.00,B,34.975000,-79.195000,NC,None,None,11,Friday,NaN,hoke,37093,Arson,0.000
331927,340162,FED,BLM,2003,2003-07-09,190,1146,5.0,Debris Burning,3.00,B,39.678800,-115.908400,NV,Eureka,011,7,Wednesday,1146.0,eureka,Eureka,Accident,1.000
1811170,300226693,NONFED,ST/C&L,2015,2015-05-04,124,1735,2.0,Equipment Use,0.25,A,42.195607,-72.603631,MA,Hampden,013,5,Monday,1735.0,hampden,Hampden,Accident,0.000
1010839,1140225,NONFED,ST/C&L,1994,1994-03-08,67,None,3.0,Smoking,2.00,B,36.178300,-80.515000,NC,None,None,3,Tuesday,NaN,yadkin,37197,Accident,0.000


In [106]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Accident', 'Prev_Accident_Fires')

CPU times: user 51.5 s, sys: 0 ns, total: 51.5 s
Wall time: 51.5 s


In [111]:
work_samp[work_samp['COUNTY2'] == 'collin']

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause,Prev_Lightning_Fires,Prev_Accident_Fires
652358,704988,NONFED,ST/C&L,2008,2008-11-01,306,None,5.0,Debris Burning,8.0,B,33.326290,-96.337360,TX,Collin,085,11,Saturday,NaN,collin,Collin,Accident,0.0,0.0
1558735,201416903,NONFED,ST/C&L,2011,2011-07-26,207,None,2.0,Equipment Use,4.0,B,33.287633,-96.745434,TX,Collin,085,7,Tuesday,NaN,collin,Collin,Accident,0.0,1.0
1562024,201420784,NONFED,ST/C&L,2011,2011-09-21,264,None,9.0,Miscellaneous,2.0,B,33.380354,-96.516091,TX,Collin,085,9,Wednesday,NaN,collin,Collin,Other,0.0,2.0
1631304,201703946,NONFED,ST/C&L,2012,2012-06-28,180,None,9.0,Miscellaneous,1.0,B,33.330126,-96.687770,TX,Collin,085,6,Thursday,NaN,collin,Collin,Other,0.0,2.0
1763570,300106729,NONFED,ST/C&L,2014,2014-07-05,186,None,5.0,Debris Burning,2.0,B,33.086948,-96.381350,TX,Collin,085,7,Saturday,NaN,collin,Collin,Accident,0.0,1.5


In [109]:
work_samp.sort_values('DISCOVERY_DATE', inplace=True)

In [112]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Arson', 'Prev_Arson_Fires')

CPU times: user 53.4 s, sys: 0 ns, total: 53.4 s
Wall time: 53.4 s


In [113]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'Cause', 'Other', 'Prev_Other_Fires')

CPU times: user 52.8 s, sys: 0 ns, total: 52.8 s
Wall time: 52.7 s


In [129]:
work_samp[work_samp['COUNTY2'] == 'collin']

,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires
652358,704988,NONFED,ST/C&L,2008,2008-11-01,306,None,5.0,Debris Burning,8.0,B,33.326290,-96.337360,TX,Collin,085,11,Saturday,NaN,collin,Collin,Accident,0.0,0.0,0.0,0.0
1558735,201416903,NONFED,ST/C&L,2011,2011-07-26,207,None,2.0,Equipment Use,4.0,B,33.287633,-96.745434,TX,Collin,085,7,Tuesday,NaN,collin,Collin,Accident,0.0,1.0,0.0,0.0
1562024,201420784,NONFED,ST/C&L,2011,2011-09-21,264,None,9.0,Miscellaneous,2.0,B,33.380354,-96.516091,TX,Collin,085,9,Wednesday,NaN,collin,Collin,Other,0.0,2.0,0.0,0.0
1631304,201703946,NONFED,ST/C&L,2012,2012-06-28,180,None,9.0,Miscellaneous,1.0,B,33.330126,-96.687770,TX,Collin,085,6,Thursday,NaN,collin,Collin,Other,0.0,2.0,0.0,1.0
1763570,300106729,NONFED,ST/C&L,2014,2014-07-05,186,None,5.0,Debris Burning,2.0,B,33.086948,-96.381350,TX,Collin,085,7,Saturday,NaN,collin,Collin,Accident,0.0,1.5,0.0,2.0


In [ ]:
# Prev fires of various class sizes

In [130]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'A', 'Prev_A_Fires')

CPU times: user 52.4 s, sys: 0 ns, total: 52.4 s
Wall time: 52.3 s


In [131]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'B', 'Prev_B_Fires')

CPU times: user 50.9 s, sys: 28 ms, total: 50.9 s
Wall time: 50.9 s


In [132]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'C', 'Prev_C_Fires')

CPU times: user 53.9 s, sys: 0 ns, total: 53.9 s
Wall time: 53.9 s


In [133]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'D', 'Prev_D_Fires')

CPU times: user 55.8 s, sys: 0 ns, total: 55.8 s
Wall time: 55.8 s


In [134]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'E', 'Prev_E_Fires')

CPU times: user 56.1 s, sys: 8 ms, total: 56.1 s
Wall time: 56.1 s


In [135]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'F', 'Prev_F_Fires')

CPU times: user 56.3 s, sys: 16 ms, total: 56.3 s
Wall time: 56.3 s


In [136]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires_filter, 'DISCOVERY_DATE', 'FIRE_SIZE_CLASS', 'G', 'Prev_G_Fires')

CPU times: user 56.2 s, sys: 4 ms, total: 56.2 s
Wall time: 56.2 s


In [139]:
work_samp.shape

(10000, 33)

## Previous fires: use for things you know a time of discovery(discovery month, day of week)

In [140]:
#use
def prev_fires(group, time_col, new_col_name, year_interval=5):
    for idx, row in group.iterrows():
        past = group[time_col] < row[time_col]
        df = group[past] #make dataframe of past fires relative to current observation
        
        #df['yr_intervals'] = (row[time_col] - df[time_col]) // year_interval #if using 'FIRE_YEAR' col
        #print(df['yr_intervals'])
        df['yr_intervals'] = (row[time_col] - df[time_col]) // np.timedelta64(year_interval, 'Y') #how many 5-years have passed?
        df['weight'] = 1 / 2**(df['yr_intervals']) 
                #weight of 1 for within past 5 years, 0.5 for 5-10 years, 0.25 for 10-15 years, etc
        #df['prev_fires'] = df['weight'].sum()
        #print(df['prev_fires'])
        count_fires = df['weight'].sum()
        #print(count_fires)
        #count_fires = df['prev_fires'].iloc[0]
        group.loc[idx,new_col_name] = count_fires
    return group
        

In [141]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2']).apply(prev_fires, 'DISCOVERY_DATE', 'Prev_Fires_Same_Month') #actually the number of fires at same location, weighed by years passed

CPU times: user 44.8 s, sys: 0 ns, total: 44.8 s
Wall time: 44.8 s


In [142]:
%time work_samp = work_samp.groupby(['STATE', 'COUNTY2', 'DayofWeek']).apply(prev_fires, 'DISCOVERY_DATE', 'Prev_Fires_Same_DayofWeek')#duplicated

CPU times: user 44.8 s, sys: 4 ms, total: 44.8 s
Wall time: 44.8 s


,FOD_ID,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,Month,DayofWeek,DISCOVERY_TIME2,COUNTY2,COUNTY_ID,Cause,Prev_Lightning_Fires,Prev_Accident_Fires,Prev_Arson_Fires,Prev_Other_Fires,Prev_A_Fires,Prev_B_Fires,Prev_C_Fires,Prev_D_Fires,Prev_E_Fires,Prev_F_Fires,Prev_G_Fires,Prev_Fires_Same_Month,Prev_Fires_Same_DayofWeek
1849538,300276406,NONFED,ST/C&L,2015,2015-12-16,350,1535,9.0,Miscellaneous,10.00,C,27.822000,-81.974600,FL,Polk,105,12,Wednesday,1535.0,polk,Polk,Other,2.0,4.875,1.0000,1.75,1.375,6.6250,1.125,0.0,0.5,0.0,0.0,9.6250,9.6250
1829649,300249480,NONFED,ST/C&L,2015,2015-12-16,350,1355,5.0,Debris Burning,0.01,A,38.340000,-98.200000,KS,Rice,159,12,Wednesday,1355.0,rice,Rice,Accident,0.0,0.000,0.0000,0.00,0.000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0000,0.0000
1809490,300224607,NONFED,ST/C&L,2015,2015-12-26,360,None,11.0,Powerline,7.00,B,35.812978,-101.427532,TX,Hutchinson,233,12,Saturday,NaN,hutchinson,Hutchinson,Accident,0.0,0.000,0.0625,0.50,0.000,0.5625,0.000,0.0,0.0,0.0,0.0,0.5625,0.5625
1851245,300280374,NONFED,ST/C&L,2015,2015-12-26,360,None,9.0,Miscellaneous,25.00,C,35.632331,-101.408358,TX,Hutchinson,233,12,Saturday,NaN,hutchinson,Hutchinson,Other,0.0,0.000,0.0625,0.50,0.000,0.5625,0.000,0.0,0.0,0.0,0.0,0.5625,0.5625
1857735,300291485,NONFED,ST/C&L,2015,2015-12-29,363,1256,11.0,Powerline,0.06,A,35.291117,-83.485833,NC,Macon,113,12,Tuesday,1256.0,macon,Macon,Accident,0.0,1.625,0.0625,1.25,2.250,0.6875,0.000,0.0,0.0,0.0,0.0,2.9375,2.9375


In [146]:
with open('fire_10000_feats.pkl', 'wb') as picklefile:
    pickle.dump(work_samp, picklefile)